In [6]:
#Importing Libraries
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import pandas as pd
from IPython.display import FileLink
import geopandas as gpd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import paramiko
from io import StringIO
from shapely.geometry import MultiPoint, MultiPolygon
import scipy
import scipy.cluster
from shapely.ops import unary_union
import calendar
from datetime import datetime
from geopy.distance import great_circle
from scipy.optimize import minimize

/Users/rajatsansaniwal/git_geoboards/geoboards/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
#Establishing connection with datawarehouse
def get_conn_postgres(SSH_required,key_path):

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [8]:
#Getting dataframe from datawarehouse
def get_df_from_sql_postgres(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [10]:
#Getting Ops Main Data for this and previous month
SSH_required = 'Yes'
key_path = '/Users/rajatsansaniwal/Documents/tunnel-ssh .cer'

query = "select warehouse_name, last_mile_hub, count(distinct awb) as orders from public.ops_main where date_trunc('month', created_date) >= date_trunc('month', now() - interval'1 month') and shipping_partner = 'Hyperlocal' and shipping_city = 'Bangalore' and warehouse_city = 'Bangalore' group by warehouse_name, last_mile_hub;"
# Establish a connection
conn = get_conn_postgres(SSH_required, key_path)

# Retrieve data into a DataFrame
df_ops_main = get_df_from_sql_postgres(SSH_required, query, key_path)

# Now you can perform further operations with the DataFrame
print(df_ops_main.head())

Tunnel Started
Connection Made
                                      warehouse_name last_mile_hub  orders
0  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          BLDR      28
1  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          CMRJ       3
2  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          ECTY      15
3  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          HBBL       9
4  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          JPNR      20


In [11]:
#Getting client warehouse lat longs for BLR
df_warehouse = pd.read_csv('warehouses_w_lat_lng_blr.csv')
df_warehouse = df_warehouse.drop_duplicates(subset=['warehouse_name', 'warehouse_pincode'])
df_warehouse['lat_long'] = df_warehouse['lat_long'].apply(eval)
# print(df_warehouse.head())

# Extract latitude and longitude values from lat_long column
df_warehouse['latitude'] = df_warehouse['lat_long'].apply(lambda x: x[0])
df_warehouse['longitude'] = df_warehouse['lat_long'].apply(lambda x: x[1])

# print(df_warehouse[['latitude', 'longitude']])

In [12]:
#Getting hub lat longs for BLR
df_hubs = pd.read_csv('hubs_w_lat_lng_blr.csv')
df_hubs['last_mile_hub'] = df_hubs['sort_codes'].str.split('/').str[1].str.strip()
df_hubs['lat_long'] = df_hubs['lat_long'].apply(eval)


# Extract latitude and longitude values from lat_long column
df_hubs['latitude'] = df_hubs['lat_long'].apply(lambda x: x[0])
df_hubs['longitude'] = df_hubs['lat_long'].apply(lambda x: x[1])

# print(df_hubs[['latitude', 'longitude']])

In [13]:
# Adding lat long data to Ops Main, making dataframe 'orders'

#warehouse lat long
df_merged = pd.merge(df_ops_main, df_warehouse[['warehouse_name', 'lat_long']], on='warehouse_name', how='left')
df_merged.rename(columns={'lat_long': 'warehouse_lat_long'}, inplace=True)
# print(df_merged.head())

# hub lat long
orders = pd.merge(df_merged, df_hubs[['last_mile_hub', 'lat_long']], on='last_mile_hub', how='left')
orders.rename(columns={'lat_long': 'hub_lat_long'}, inplace=True)

print(orders.head())
print(len(orders))

                                      warehouse_name last_mile_hub  orders  \
0  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          BLDR      28   
1  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          CMRJ       3   
2  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          ECTY      15   
3  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          HBBL       9   
4  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          JPNR      20   

         warehouse_lat_long                     hub_lat_long  
0  [13.0692593, 77.7982428]         [12.9394122, 77.6921294]  
1  [13.0692593, 77.7982428]         [12.9467756, 77.5519109]  
2  [13.0692593, 77.7982428]         [12.8307773, 77.6612892]  
3  [13.0692593, 77.7982428]          [13.0405585, 77.595625]  
4  [13.0692593, 77.7982428]  [12.9144032, 77.59958150000001]  
430


In [15]:
#Analysing Warehouse Lat Long Data
other_datatypes = set()
for coords in orders['warehouse_lat_long']:
    datatype = type(coords)
    if datatype != list:
        other_datatypes.add(datatype)

print(other_datatypes)

float_values = orders[orders['warehouse_lat_long'].apply(lambda x: isinstance(x, float))]
print(float_values['warehouse_lat_long'])

# Filter the DataFrame to only include rows where warehouse_lat_long is NaN
nan_values = orders[orders['warehouse_lat_long'].apply(lambda x: isinstance(x, float) and pd.isna(x))]
print(nan_values['warehouse_name'].unique())

# Filter the DataFrame to only include rows where warehouse_lat_long is NaN
nan_values = orders[orders['warehouse_lat_long'].apply(lambda x: isinstance(x, float) and pd.isna(x))]

# Get unique warehouse names
unique_warehouse_names = nan_values['warehouse_name'].unique()

# Create a DataFrame with unique warehouse names
unique_warehouse_df = pd.DataFrame({'warehouse_name': unique_warehouse_names})

# Save the DataFrame to an Excel file
unique_warehouse_df.to_excel('not_in_warehouses_warehouse.xlsx', index=False)



{<class 'float'>}
27     NaN
28     NaN
29     NaN
30     NaN
31     NaN
32     NaN
217    NaN
218    NaN
219    NaN
220    NaN
221    NaN
397    NaN
398    NaN
399    NaN
400    NaN
401    NaN
402    NaN
403    NaN
404    NaN
405    NaN
Name: warehouse_lat_long, dtype: object
['48,1st Cross, Chowdappa layout, Opp  Federal Bank, ,K Narayanapura main road, Thanisandra, SRK Nagar Post,Bengaluru,KA'
 'Katha no 461 100 7, Comprised of Converted Survey No 100 4,'
 'SY  No  99 1, Aduru Village,']


In [16]:
# Cleaning Warehouse Lat Long Data, for non-NaN values in the warehouse_lat_long column
filtered_orders = orders.dropna(subset=['warehouse_lat_long'])
# Save the filtered DataFrame to a new variable named "orders"
orders = filtered_orders.copy()
print(len(orders))

410


In [17]:
##Analysing Hub Lat Long Data
other_datatypes = set()
for coords in orders['hub_lat_long']:
    datatype = type(coords)
    if datatype != list:
        other_datatypes.add(datatype)

print(other_datatypes)

float_values = orders[orders['hub_lat_long'].apply(lambda x: isinstance(x, float))]
print(float_values['hub_lat_long'])

# Filter the DataFrame to only include rows where hub_lat_long is NaN
nan_values = orders[orders['hub_lat_long'].apply(lambda x: isinstance(x, float) and pd.isna(x))]
print(nan_values['last_mile_hub'].unique())

# Filter the DataFrame to only include rows where hub_lat_long is NaN
nan_values = orders[orders['hub_lat_long'].apply(lambda x: isinstance(x, float) and pd.isna(x))]

# Get unique warehouse names
unique_warehouse_names = nan_values['last_mile_hub'].unique()

# Create a DataFrame with unique warehouse names
unique_warehouse_df = pd.DataFrame({'last_mile_hub': unique_warehouse_names})

# # Save the DataFrame to an Excel file
# unique_warehouse_df.to_excel('not_in_warehouses_warehouse.xlsx', index=False)



{<class 'float'>}
15     NaN
39     NaN
67     NaN
86     NaN
96     NaN
115    NaN
125    NaN
135    NaN
150    NaN
173    NaN
183    NaN
193    NaN
203    NaN
213    NaN
228    NaN
238    NaN
257    NaN
267    NaN
277    NaN
287    NaN
297    NaN
316    NaN
326    NaN
336    NaN
346    NaN
356    NaN
366    NaN
412    NaN
426    NaN
Name: hub_lat_long, dtype: object
['MTH']


In [18]:
# Cleaning Hub Lat Long Data, for non-NaN values in the hub_lat_long column
filtered_orders = orders.dropna(subset=['hub_lat_long'])

# Save the filtered DataFrame to a new variable named "orders"
orders = filtered_orders.copy()

print(orders)
print(len(orders))

                                        warehouse_name last_mile_hub  orders  \
0    15 Ground Floor, SY No  131 3, Hoskote, Anjane...          BLDR      28   
1    15 Ground Floor, SY No  131 3, Hoskote, Anjane...          CMRJ       3   
2    15 Ground Floor, SY No  131 3, Hoskote, Anjane...          ECTY      15   
3    15 Ground Floor, SY No  131 3, Hoskote, Anjane...          HBBL       9   
4    15 Ground Floor, SY No  131 3, Hoskote, Anjane...          JPNR      20   
..                                                 ...           ...     ...   
424                                   Vaaree Warehouse          JPNR      53   
425                                   Vaaree Warehouse          MRTH      64   
427                                   Vaaree Warehouse          STNG       6   
428                                   Vaaree Warehouse          UTTR      32   
429                                   Vaaree Warehouse          YLHK       9   

                 warehouse_lat_long    

In [19]:
#Defining the Fixed Motherhub

# Retrieve lat_long column for central_hub nodes
fixed_motherhub = df_hubs[df_hubs['node_type'] == 'central_hub']['lat_long']

# Access latitude and longitude values from the first row
fixed_motherhub_latitude = fixed_motherhub.iloc[0][0]
fixed_motherhub_longitude = fixed_motherhub.iloc[0][1]

# Create a tuple with latitude and longitude values
fixed_motherhub = (fixed_motherhub_latitude, fixed_motherhub_longitude)

# Print the fixed_motherhub coordinates
print(fixed_motherhub)

(12.9497375, 77.6982656)


In [20]:
# Group by last_mile_hub and hub_lat_long, and sum the orders
hub_data = orders.groupby(['last_mile_hub'])['orders'].sum().reset_index()

# Rename columns appropriately
hub_data.columns = ['last_mile_hub', 'sum_of_orders']

df_hubs = pd.merge(hub_data, df_hubs, on='last_mile_hub', how='left')
print(df_hubs.head())


  last_mile_hub  sum_of_orders  node_id  warehouse_id       node_name  \
0          BLDR          19102       33           182  BLDR-Franchise   
1          CMRJ           7186       32           182  CMRJ-Franchise   
2          ECTY          20108       10           182    BLR FRH ECTY   
3          HBBL          29638        7           182     DS BLR HBBL   
4          JPNR          33947        2           182     DS BLR BOMM   

   location_id      team_names  fuel_rate  distance_limit sort_codes  \
0            1    BLR_FRH_BLDR          0               0   BLR/BLDR   
1            1    BLR_FRH_CMRJ          0               0   BLR/CMRJ   
2            1    BLR_FRH_ECTY          0             100   BLR/ECTY   
3            1  GS_BLR_LM_HBBL          3             100   BLR/HBBL   
4            2          blr-lm          3             100   BLR/JPNR   

                                             address   contact_name  \
0  22/1A, 22/1B, Kariyammana Agrahara Rd, Kadubee...  Bis

In [21]:
# Group by last_mile_hub and hub_lat_long, and sum the orders
warehouse_data = orders.groupby(['warehouse_name'])['orders'].sum().reset_index()

# Rename columns appropriately
warehouse_data.columns = ['warehouse_name', 'sum_of_orders']

df_warehouse = pd.merge(warehouse_data, df_warehouse, on='warehouse_name', how='left')
print(df_warehouse.head())


                                      warehouse_name  sum_of_orders  \
0  15 Ground Floor, SY No  131 3, Hoskote, Anjane...            104   
1     16   1, Deganhalli Village Road, Kasaba Hobli,             10   
2           24 2, Chikkahullur Village, Kasba Hobli,           8514   
3  48,1st Cross, Chowdappa layout,,Opp  Federal B...             80   
4                                       Anubhava BLR           2046   

    warehouse_city                                  warehouse_address  \
0        Bangalore  15 Ground Floor, SY No  131 3, Hoskote, Anjane...   
1  Bangalore Rural  16   1, Deganhalli Village Road, Kasaba Hobli,...   
2  Bangalore Rural  24 2, Chikkahullur Village, Kasba Hobli, Shidl...   
3        BANGALORE  48,1st Cross, Chowdappa layout,,Opp  Federal B...   
4        Bangalore  Survey No 78/10, A2, Bellary Rd, Chikkajala, B...   

   warehouse_pincode                        lat_long   latitude  longitude  
0             560067        [13.0692593, 77.7982428]  13.

In [22]:
#cost should not go above a limit
#cost can be defined with rwith centre as centre

In [23]:
# Total Distance Currently i.e. only Fixed Motherhub exists
def total_distance_fixed_only(orders, fixed_motherhub):
    total_dist = 0

    for _, row in orders.iterrows():
        client_latitude = row['warehouse_lat_long'][0]
        client_longitude = row['warehouse_lat_long'][1]
        client_location = (client_latitude, client_longitude)

        last_mile_latitude = row['hub_lat_long'][0]
        last_mile_longitude = row['hub_lat_long'][1]
        last_mile_location = (last_mile_latitude, last_mile_longitude)
        
        dist_client_to_fixed = great_circle(client_location, fixed_motherhub).km
        dist_fixed_to_last_mile = great_circle(fixed_motherhub, last_mile_location).km
        total_dist_fixed = dist_client_to_fixed + dist_fixed_to_last_mile
        
        total_dist += row['orders'] * total_dist_fixed
    
    return total_dist

# Calculate the total distance with only the fixed motherhub
total_distance_current = total_distance_fixed_only(orders, fixed_motherhub)
print("Total Distance with Only Fixed Motherhub:", total_distance_current)

Total Distance with Only Fixed Motherhub: 6414514.831016428


In [24]:
# Optimum Location considering only the new motherhub will operate and fixed motherhub will be obsolete

In [25]:
#Total Distance function for fixed motherhub and one new motherhub
def total_distance(new_motherhub, orders, fixed_motherhubs):
    total_dist = 0

    for _, row in orders.iterrows():
        client_latitude = row['warehouse_lat_long'][0]
        client_longitude = row['warehouse_lat_long'][1]
        client_location = (client_latitude, client_longitude)

        last_mile_latitude = row['hub_lat_long'][0]
        last_mile_longitude = row['hub_lat_long'][1]
        last_mile_location = (last_mile_latitude, last_mile_longitude)
        
        for fixed_motherhub in fixed_motherhubs:
            dist_client_to_fixed = great_circle(client_location, fixed_motherhub).km
            dist_fixed_to_last_mile = great_circle(fixed_motherhub, last_mile_location).km
            total_dist_fixed = dist_client_to_fixed + dist_fixed_to_last_mile
            
            dist_client_to_new = great_circle(client_location, tuple(new_motherhub)).km
            dist_new_to_last_mile = great_circle(tuple(new_motherhub), last_mile_location).km
            total_dist_new = dist_client_to_new + dist_new_to_last_mile
            
            total_dist += row['orders'] * min(total_dist_fixed, total_dist_new)
    
    return total_dist

# Calculate the mean latitude and longitude separately
mean_latitude = orders['warehouse_lat_long'].apply(lambda x: x[0]).mean()
mean_longitude = orders['warehouse_lat_long'].apply(lambda x: x[1]).mean()

# Create the initial guess tuple
initial_guess = (mean_latitude, mean_longitude)
print("Initial Guess:", initial_guess)

# Calculate the initial total distance with the initial guess
initial_total_dist = total_distance(initial_guess, orders, [fixed_motherhub])
print("Initial Total Distance:", initial_total_dist)

# Apply the optimization
result = minimize(total_distance, initial_guess, args=(orders, [fixed_motherhub]), method='Nelder-Mead')

# Print the result
# print("Optimization Result:", result)
optimal_new_motherhub_location = result.x
print("Optimal New Motherhub Location:", optimal_new_motherhub_location)
print("Total Distance:", result.fun)

Initial Guess: (np.float64(13.048968657480318), np.float64(77.56845083438319))
Initial Total Distance: 5076559.63912762
Optimal New Motherhub Location: [12.9856635  77.55718145]
Total Distance: 4929499.110118498


In [26]:
# Looking at the Mother Hub assigned

# Access latitude and longitude values of new motherhub
optimal_new_motherhub_latitude = optimal_new_motherhub_location[0]
optimal_new_motherhub_longitude = optimal_new_motherhub_location[1]

# Create a tuple with latitude and longitude values
new_motherhub = (optimal_new_motherhub_latitude, optimal_new_motherhub_longitude)

motherhub_labels = []

# Loop through each order and determine whether it passes through the fixed or new motherhub
for _, row in orders.iterrows():
    client_latitude = row['warehouse_lat_long'][0]
    client_longitude = row['warehouse_lat_long'][1]
    client_location = (client_latitude, client_longitude)

    last_mile_latitude = row['hub_lat_long'][0]
    last_mile_longitude = row['hub_lat_long'][1]
    last_mile_location = (last_mile_latitude, last_mile_longitude)
    
    dist_client_to_fixed = great_circle(client_location, fixed_motherhub).km
    dist_fixed_to_last_mile = great_circle(fixed_motherhub, last_mile_location).km
    total_dist_fixed = dist_client_to_fixed + dist_fixed_to_last_mile
    
    dist_client_to_new = great_circle(client_location, tuple(new_motherhub)).km
    dist_new_to_last_mile = great_circle(tuple(new_motherhub), last_mile_location).km
    total_dist_new = dist_client_to_new + dist_new_to_last_mile
    
    if total_dist_fixed < total_dist_new:
        motherhub_labels.append('Fixed')
    else:
        motherhub_labels.append('New')

# Add the list of labels as a new column in the orders DataFrame
orders['motherhub_label'] = motherhub_labels


In [27]:
# Group by 'motherhub_label' and calculate the sum of orders for each group
orders_grouped = orders.groupby('motherhub_label')['orders'].sum().reset_index()

# Calculate the total sum of orders
total_orders = orders['orders'].sum()

# Calculate the percentage of total sum of orders for each group
orders_grouped['percentage_of_total'] = (orders_grouped['orders'] / total_orders) * 100

# Convert the percentage to integer
orders_grouped['percentage_of_total'] = orders_grouped['percentage_of_total'].astype(int)

print(orders_grouped)


  motherhub_label  orders  percentage_of_total
0           Fixed   62379                   38
1             New   99164                   61


In [28]:
# Specify the file path
file_path = "orders.xlsx"

# Write the DataFrame to an Excel file
orders.to_excel(file_path, index=False)

In [29]:
# Combine the data from both maps into a single figure
combined_figure = {
    'data': [
        # Trace for optimal new motherhub location
        {
            'type': 'scattermapbox',
            'lat': [optimal_new_motherhub_location[0]],  # Optimal new motherhub latitude
            'lon': [optimal_new_motherhub_location[1]],  # Optimal new motherhub longitude
            'mode': 'markers',
            'marker': {
                'size': 15,
                'color': 'purple'  # Set color for optimal new motherhub location
            },
            'name': 'Optimal New Motherhub Location'
        }
        ,
        # Traces from the second map (df_hubs)
        *px.scatter_mapbox(
            df_hubs,
            lat='latitude',
            lon='longitude',
            hover_name='last_mile_hub',
            hover_data={'latitude': False, 'longitude': False},
            color='node_type',  # Map node_type to marker color
            color_discrete_map={
                'central_hub': 'green',    # Set green color for central_hub
                'lm_hub': 'blue',          # Set blue color for lm_hub
                'franchise_hub': 'red'  # Set dark yellow color for franchise_hub
            },
            zoom=10,
            height=600
        ).update_traces(marker=dict(size=12)).to_dict()['data']
    ],
    'layout': {
        'mapbox': {
            'style': "open-street-map",
            'center': {'lat': 12.97, 'lon': 77.59},  # Center coordinates of Bangalore
            'zoom': 10
        }
    }
}

# Initialize the app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1("A Fixed and A New Motherhub Functioning", style={'text-align': 'center'}),
    dcc.Graph(
        id='map',
        figure=combined_figure,
        style={'height': '90vh'}  # Set the height of the graph to 90% of the viewport height
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8052)



In [30]:
# Define the total_distance function considering only 1 new motherhub
def total_distance(new_motherhub, orders):
    total_dist = 0

    for _, row in orders.iterrows():
        client_latitude = row['warehouse_lat_long'][0]
        client_longitude = row['warehouse_lat_long'][1]
        client_location = (client_latitude, client_longitude)

        last_mile_latitude = row['hub_lat_long'][0]
        last_mile_longitude = row['hub_lat_long'][1]
        last_mile_location = (last_mile_latitude, last_mile_longitude)
        
        dist_client_to_new = great_circle(client_location, tuple(new_motherhub)).km
        dist_new_to_last_mile = great_circle(tuple(new_motherhub), last_mile_location).km
        
        total_dist += row['orders'] * (dist_client_to_new + dist_new_to_last_mile)
    
    return total_dist

print("Initial Guess:", initial_guess)

# Calculate the initial total distance with the initial guess
initial_total_dist = total_distance(initial_guess, orders)
print("Initial Total Distance:", initial_total_dist)

# Apply the optimization
result = minimize(total_distance, initial_guess, args=(orders,), method='Nelder-Mead')

# Print the result
optimal_new_motherhub_location = result.x
print("Optimal New Motherhub Location:", optimal_new_motherhub_location)
print("Total Distance:", result.fun)


Initial Guess: (np.float64(13.048968657480318), np.float64(77.56845083438319))
Initial Total Distance: 5850869.677342431
Optimal New Motherhub Location: [12.96747282 77.58994939]
Total Distance: 5324946.460161096


In [31]:
import plotly.graph_objects as go

In [32]:
# Aggregate hover text for warehouses with the same latitude and longitude
df_warehouse['hover_text'] = (
    'Warehouse: ' + df_warehouse['warehouse_name'] + '<br>' +
    'Orders: ' + df_warehouse['sum_of_orders'].astype(str)
)

aggregated_data = df_warehouse.groupby(['latitude', 'longitude']).agg({
    'hover_text': '<br>'.join,
    'sum_of_orders': 'sum'
}).reset_index()

# Scale the order sizes for better visualization
order_sizes = aggregated_data['sum_of_orders'] / aggregated_data['sum_of_orders'].max() * 10

# Create the combined figure
combined_figure = go.Figure()

# Trace for optimal new motherhub location
combined_figure.add_trace(go.Scattermapbox(
    lat=[optimal_new_motherhub_location[0]],
    lon=[optimal_new_motherhub_location[1]],
    mode='markers',
    marker={
        'size': 20,
        'color': 'purple',
        'opacity': 1  # Adjust opacity for visibility
    },
    name='Optimal New Motherhub Location'
))

# Traces from the second map (df_hubs)
hub_traces = px.scatter_mapbox(
    df_hubs,
    lat='latitude',
    lon='longitude',
    hover_name='node_name',
    hover_data={'latitude': False, 'longitude': False},
    color='node_type',
    color_discrete_map={
        'central_hub': 'green',
        'lm_hub': 'blue',
        'franchise_hub': 'blue'
    },
    zoom=10,
    height=600
).update_traces(marker=dict(size=15, opacity=1)).to_dict()['data']

for trace in hub_traces:
    combined_figure.add_trace(trace)

# Trace for warehouses heatmap
combined_figure.add_trace(go.Densitymapbox(
    lat=aggregated_data['latitude'],
    lon=aggregated_data['longitude'],
    z=aggregated_data['sum_of_orders'],
    radius=150,  # Adjusted radius
    opacity=1,  # Adjust opacity for better visibility
    colorscale='Hot',
    showscale=False,
    name='Warehouses Heatmap'
))

# Scattermapbox trace for hover information
combined_figure.add_trace(go.Scattermapbox(
    lat=aggregated_data['latitude'],
    lon=aggregated_data['longitude'],
    mode='markers',
    marker=go.scattermapbox.Marker(size=10, color='black', opacity=1),  # Transparent markers for hover
    text=aggregated_data['hover_text'],
    hoverinfo='text',
    name='Warehouses'
))

combined_figure.update_layout(
    mapbox={
        'style': "open-street-map",
        'center': {'lat': 12.97, 'lon': 77.59},  # Center coordinates of Bangalore
        'zoom': 9
    },
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},  # Remove margins to fit the map better
    height=800  # Set the height of the map
)

# Initialize the app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1("Only One New Motherhub Functioning", style={'text-align': 'center'}),
    dcc.Graph(
        id='map',
        figure=combined_figure,
        style={'height': '90vh'}  # Set the height of the graph to 90% of the viewport height
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8053)

In [33]:
def total_distance(new_motherhubs, orders):
    total_dist = 0
    new_motherhub1 = (new_motherhubs[0], new_motherhubs[1])
    new_motherhub2 = (new_motherhubs[2], new_motherhubs[3])

    for _, row in orders.iterrows():
        client_latitude = row['warehouse_lat_long'][0]
        client_longitude = row['warehouse_lat_long'][1]
        client_location = (client_latitude, client_longitude)

        last_mile_latitude = row['hub_lat_long'][0]
        last_mile_longitude = row['hub_lat_long'][1]
        last_mile_location = (last_mile_latitude, last_mile_longitude)

        dist_client_to_new1 = great_circle(client_location, new_motherhub1).km
        dist_new1_to_last_mile = great_circle(new_motherhub1, last_mile_location).km
        total_dist_new1 = dist_client_to_new1 + dist_new1_to_last_mile

        dist_client_to_new2 = great_circle(client_location, new_motherhub2).km
        dist_new2_to_last_mile = great_circle(new_motherhub2, last_mile_location).km
        total_dist_new2 = dist_client_to_new2 + dist_new2_to_last_mile

        total_dist += row['orders'] * min(total_dist_new1, total_dist_new2)
    
    return total_dist

# Calculate the mean latitude and longitude separately for initial guess
mean_latitude = orders['warehouse_lat_long'].apply(lambda x: x[0]).mean()
mean_longitude = orders['warehouse_lat_long'].apply(lambda x: x[1]).mean()

# Create the initial guess tuple for two motherhubs
initial_guess = (mean_latitude, mean_longitude, mean_latitude, mean_longitude)
print("Initial Guess:", initial_guess)

# Calculate the initial total distance with the initial guess
initial_total_dist = total_distance(initial_guess, orders)
print("Initial Total Distance:", initial_total_dist)

# Apply the optimization for two new motherhubs
result = minimize(total_distance, initial_guess, args=(orders,), method='Nelder-Mead')

# Print the result
optimal_new_motherhub_location1 = result.x[:2]
optimal_new_motherhub_location2 = result.x[2:]
print("Optimal New Motherhub Locations:", optimal_new_motherhub_location1, optimal_new_motherhub_location2)
print("Total Distance:", result.fun)

# Combine the data from both maps into a single figure
combined_figure = {
    'data': [
        # Trace for optimal new motherhub location 1
        {
            'type': 'scattermapbox',
            'lat': [optimal_new_motherhub_location1[0]],  # Optimal new motherhub latitude
            'lon': [optimal_new_motherhub_location1[1]],  # Optimal new motherhub longitude
            'mode': 'markers',
            'marker': {
                'size': 20,
                'color': 'purple'  # Set color for optimal new motherhub location 1
            },
            'name': 'Optimal New Motherhub Location 1'
        },
        # Trace for optimal new motherhub location 2
        {
            'type': 'scattermapbox',
            'lat': [optimal_new_motherhub_location2[0]],  # Optimal new motherhub latitude
            'lon': [optimal_new_motherhub_location2[1]],  # Optimal new motherhub longitude
            'mode': 'markers',
            'marker': {
                'size': 20,
                'color': 'orange'  # Set color for optimal new motherhub location 2
            },
            'name': 'Optimal New Motherhub Location 2'
        },
        # Traces from the second map (df_hubs)
        *px.scatter_mapbox(
            df_hubs.dropna(subset=['latitude', 'longitude']),
            lat='latitude',
            lon='longitude',
            hover_name='node_name',
            hover_data={'latitude': False, 'longitude': False},
            color='node_type',  # Map node_type to marker color
            color_discrete_map={
                'central_hub': 'green',    # Set green color for central_hub
                'lm_hub': 'blue',          # Set blue color for lm_hub
                'franchise_hub': 'red'  # Set dark yellow color for franchise_hub
            },
            zoom=10,
            height=600
        ).update_traces(marker=dict(size=12)).to_dict()['data']
    ],
    'layout': {
        'mapbox': {
            'style': "open-street-map",
            'center': {'lat': 12.97, 'lon': 77.59},  # Center coordinates of Bangalore
            'zoom': 10
        }
    }
}

# Initialize the app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1("Two New Motherhubs Functioning", style={'text-align': 'center'}),
    dcc.Graph(
        id='map',
        figure=combined_figure,
        style={'height': '90vh'}  # Set the height of the graph to 90% of the viewport height
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8054)

Initial Guess: (np.float64(13.048968657480318), np.float64(77.56845083438319), np.float64(13.048968657480318), np.float64(77.56845083438319))
Initial Total Distance: 5850869.677342431
Optimal New Motherhub Locations: [13.04019202 77.59528239] [12.9144032 77.5995815]
Total Distance: 4809511.819876614
